In [1]:
import tensorflow as tf
from pathlib import Path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Create the dataset
data_dir = Path('./dataset/mudkip/')

img_height = 200
img_width = 200

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed = 258, # mudkips pokedex number
    image_size=(img_height, img_width),
    crop_to_aspect_ratio=True,
    shuffle=True
)

valid_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed = 258, # mudkips pokedex number
    image_size=(img_height, img_width),
    crop_to_aspect_ratio=True,
    shuffle=True
)

class_names = train_ds.class_names

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
valid_ds = valid_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Create model
# TODO: mess around and learn more about the layers
# The number of classes may differ depending on the hunt and how it is compiled
# It will either be 2 (target_normal, target_shiny) or 4 (target_normal, target_shiny, not_target_normal, not_target_shiny) 
number_of_classes = len(class_names)

#data_augmentation = tf.keras.models.Sequential()
#data_augmentation.add(tf.keras.layers.RandomFlip("horizontal", input_shape=(img_height, img_width, 3)))
#data_augmentation.add(tf.keras.layers.RandomRotation(0.1))
#data_augmentation.add(tf.keras.layers.RandomZoom(0.1))

model = tf.keras.models.Sequential()
#model.add(data_augmentation)
model.add(tf.keras.layers.Rescaling(1/.255, input_shape=(img_height, img_width, 3)))
model.add(tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu')) # for mudkip used 32
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu')) # for mudkip used 64
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu')) # for mudkip used 128
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu')) # for mudkip used 256
model.add(tf.keras.layers.Dense(number_of_classes))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

In [ ]:
# Train model
epochs = 10
history = model.fit(train_ds, validation_data=valid_ds, epochs=epochs)

# Visualize training results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Test the model with a single image
test_path = Path('./test_imgs/mudkip/1.png')

img = tf.keras.utils.load_img(test_path, target_size=(img_height, img_width), keep_aspect_ratio=True)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print("This image most likely is {} with a {:.2f} percent confidence".format(class_names[np.argmax(score)], 100 * np.max(score)))
img

In [ ]:
# Test the model in a batch
# TODO: TEST ON LIVE STREAM ARCHIVES
test_path = Path('./test_imgs/mudkip/')
actual = ['target_normal', 'target_normal', 'target_normal', 'target_normal', 'target_normal',
          'target_shiny', 'target_shiny', 'target_shiny', 'target_shiny', 'target_shiny',
          'target_normal', 'target_normal', 'target_normal', 'target_normal', 'target_normal',
          'target_shiny', 'target_shiny', 'target_shiny', 'target_shiny', 'target_shiny',]
predicts = []
num_normal = 10
num_shiny = 10

for file in test_path.iterdir():
    img = tf.keras.utils.load_img(file, target_size=(img_height, img_width), keep_aspect_ratio=True)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    predicts.append(class_names[np.argmax(score)])
    print("This image most likely is {} with a {:.2f} percent confidence".format(class_names[np.argmax(score)], 100 * np.max(score)))

overall_score = 0
normal_score = 0
shiny_score = 0
for i in range(len(actual)):
    if actual[i] == predicts[i]:
        overall_score += 1
        if actual[i] == 'target_normal':
            normal_score += 1
        elif actual[i] == 'target_shiny':
            shiny_score += 1

overall_percent = (overall_score / len(actual)) * 100
normal_percent = (normal_score / num_normal) * 100
shiny_percent = (shiny_score / num_shiny) * 100

print(f"Overall Score: {overall_percent}% ({overall_score}/{len(actual)})")
print(f"Normal Score: {normal_percent}% ({normal_score}/{num_normal})")
print(f"Shiny Score: {shiny_percent}% ({shiny_score}/{num_shiny})")

In [ ]:
# Save the model
model.save('oras_starter_mudkip')